In [1]:
datos=spark.read.csv("customer_churn.csv",header=True,
                     inferSchema=True)

In [2]:
datos.show(2,vertical=True),datos.printSchema()

-RECORD 0-------------------------------
 Names           | Cameron Williams     
 Age             | 42.0                 
 Total_Purchase  | 11066.8              
 Account_Manager | 0                    
 Years           | 7.22                 
 Num_Sites       | 8.0                  
 Onboard_date    | 2013-08-30 07:00:40  
 Location        | 10265 Elizabeth M... 
 Company         | Harvey LLC           
 Churn           | 1                    
-RECORD 1-------------------------------
 Names           | Kevin Mueller        
 Age             | 41.0                 
 Total_Purchase  | 11916.22             
 Account_Manager | 0                    
 Years           | 6.5                  
 Num_Sites       | 11.0                 
 Onboard_date    | 2013-08-13 00:38:46  
 Location        | 6157 Frank Garden... 
 Company         | Wilson PLC           
 Churn           | 1                    
only showing top 2 rows

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = tr

(None, None)

In [3]:
datos.describe().show(vertical=True)

-RECORD 0-------------------------------
 summary         | count                
 Names           | 900                  
 Age             | 900                  
 Total_Purchase  | 900                  
 Account_Manager | 900                  
 Years           | 900                  
 Num_Sites       | 900                  
 Location        | 900                  
 Company         | 900                  
 Churn           | 900                  
-RECORD 1-------------------------------
 summary         | mean                 
 Names           | null                 
 Age             | 41.81666666666667    
 Total_Purchase  | 10062.82403333334    
 Account_Manager | 0.4811111111111111   
 Years           | 5.27315555555555     
 Num_Sites       | 8.587777777777777    
 Location        | null                 
 Company         | null                 
 Churn           | 0.16666666666666666  
-RECORD 2-------------------------------
 summary         | stddev               
 Names          

In [7]:
datos.select("Years").toPandas().plot.box()

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'

In [9]:
import matplotlib,pandas
matplotlib.__version__,pandas.__version__

('1.5.1', '0.23.4')

In [10]:
from pandas.plotting import scatter_matrix
g=scatter_matrix(datos.toPandas(),figsize=(10,10))

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'

In [7]:
datos.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
datos.crosstab('Account_Manager','Churn').show()

+---------------------+---+---+
|Account_Manager_Churn|  0|  1|
+---------------------+---+---+
|                    1|349| 84|
|                    0|401| 66|
+---------------------+---+---+



In [9]:
datos.groupBy("Company").count().orderBy("count",ascending=False).count()

873

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
constructor=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol="features")

In [12]:
datosF=constructor.transform(datos)

In [13]:
datosF.show(1,vertical=True)

-RECORD 0-------------------------------
 Names           | Cameron Williams     
 Age             | 42.0                 
 Total_Purchase  | 11066.8              
 Account_Manager | 0                    
 Years           | 7.22                 
 Num_Sites       | 8.0                  
 Onboard_date    | 2013-08-30 07:00:40  
 Location        | 10265 Elizabeth M... 
 Company         | Harvey LLC           
 Churn           | 1                    
 features        | [42.0,11066.8,0.0... 
only showing top 1 row



In [14]:
entrena,evalua=datosF.randomSplit([0.8,0.2])

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr=LogisticRegression(labelCol="Churn")

In [17]:
modelo=lr.fit(entrena)

In [18]:
modelo.summary.areaUnderROC

0.9080009787129927

In [19]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [20]:
prediccion=modelo.transform(evalua)

In [21]:
evaluadorB=BinaryClassificationEvaluator(labelCol="Churn")

In [22]:
evaluadorB.evaluate(prediccion)#AUC

0.9052478134110785

In [23]:
evaluadorM=MulticlassClassificationEvaluator(labelCol="Churn",
                                             metricName="accuracy")

In [24]:
evaluadorM.evaluate(prediccion)

0.88

In [25]:
modeloOK=lr.fit(datosF)

modeloOK.save("modeloLR")

# Ahora con Pipeline

In [43]:
from pyspark.ml import Pipeline

In [44]:
entrena2,evalua2=datos.randomSplit([0.8,0.2])


In [45]:
constructor=VectorAssembler(inputCols=['Age', 'Total_Purchase', 
    'Account_Manager','Years', 'Num_Sites'],outputCol="features")

In [46]:
lr=LogisticRegression(labelCol="Churn")

In [47]:
pasos=Pipeline(stages=[constructor,lr])#pasos a ejecutar en secuencia

In [48]:
modelo=pasos.fit(entrena2)

In [49]:
prediccion=modelo.transform(evalua2)

In [50]:
evaluadorB=BinaryClassificationEvaluator(labelCol="Churn")
evaluadorB.evaluate(prediccion)

0.9309868875086267